## Fo-SiO$_2$ system

This notebook generates a reaction `.rxml` file for the full Fo-SiO$_2$ system  This system is described in detail in Lucy Tweed's thesis (2021).

In [ ]:
import sympy as sym
import os
from glob import glob
import warnings
sym.init_printing()

In [ ]:
from thermocodegen.codegen import reaction
from thermocodegen.coder import coder

choose a version tag for generating different versions of Berman endmember numbers

### let's set up some directory names for clarity

In [ ]:
HOME_DIR = os.path.abspath('../../')
SPUD_DIR = HOME_DIR+'/reactions'
try:
    os.makedirs(SPUD_DIR)
except:
    pass

Set a reference string for this Notebook

In [ ]:
reference = 'fo_sio2/notebooks/Generate_fo_sio2_poly_linear_rxn.ipynb'

List of phases present in this set of reactions

In [ ]:
phase_names = ['Liquid', 'Olivine', 'Orthopyroxene', 'Silica_polymorph' ]

Path to the thermodynamic database tarball file that this set of reactions are built with

Here we use an absolute path to a local file which is safer than a relative path, but can make the rxml files harder to share

In [ ]:
#db = HOME_DIR+'/database/fo_sio2_db.tar.gz' 

Alternatively, we could use a remote thermodynamic database, for example the one available as a DOI from zenodo 

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.6350797.svg)](https://doi.org/10.5281/zenodo.6350797)


In [1]:
db = 'https://zenodo.org/record/6350797/files/fo_sio2_db.tar.gz'

Instantiate a Reaction object. This is initialized using the 'from_database' class method. Initializing in this way requires the total number of reaction (total_reactions), a list of phase names (phase_names) and a path to a thermocodegen generated thermodynamic database.

In [ ]:
rxn = reaction.Reaction.from_database(total_reactions=3, phase_names=phase_names, database=db)

The 'model_dict' attribute of this object contains the current state of the information that we have given and what has been extracted from the thermodynamic database 'fo_sio2_db.tar.gz'

In [ ]:
rxn.model_dict

#### Set some utility indices for referencing phases and endmembers (this should be automated somehow)

In [ ]:
iLq = 0
iOl = 1
iOpx = 2
iSi = 3
kSi = 0
kFo = 1

### Define the reactions in this system, and then write SymPy that describes the reactions rates

First we pull out the SymPy symbols for the independent variables (temperature T, pressure P, concentration C and phase fraction Phi). Concentration C is a sym.MatrixSymbol of dimension N (number of phases) by Kmax (maximum number of endmembers). Phase fraction Phi is a sym.MatrixSymbol of dimension N (number of phases) by 1 (i.e., a vector).

In [ ]:
# Variables
T = rxn.T
P = rxn.P
C = rxn.C
Phi = rxn.Phi

Now we pull out the special symbol reserved for the affinity, A. This is a sym.MatrixSymbol of dimension J (number of reactions) by 1.

In [ ]:
# Affinity
A = rxn.A

Now we define the parameters used in the reaction rate expressions. These should be sym.Symbol objects. We also need to create lists that contain the parameter names as strings, their units (in string form) and the corresponding SymPy symbols. Note that these need to be ordered correctly.

**Global Parameters**
* T0: reference temperature in K
* R: Gas Constant
* r0_xtal: reaction rate of crystallization (kg m$^{-2}$ s$^{-1}$)
* r0_melt: reaction rate of melting (kg m$^{-2}$ s$^{-1}$)
* Stot: total available surface area
* eps:  Phase Fraction regularization parameter such that $S = Stot\frac{\phi}{\phi + \epsilon}$
* M0: reference reaction molar mass (g/m)
* M_fo: reference molar mass fo reaction (g/m)
* M_en: reference molar mass Opx reaction (g/m)
* M_en: reference molar mass Si reaction (g/m)

In [ ]:
# Global Parameters
param_strings = ['T0', 'R', 'r0_xtal', 'r0_melt', 'Stot', 'eps']
units = ['K','J/(mol K)', 'kg/m^2/s', 'kg/m^2/s', 'm^-1','' ]

# mass parameters for reactions
param_strings += ['M0', 'M_fo', 'M_en', 'M_qz']
units += ['g/mol' for i in range(4)]

Given Parameter strings and units,  we can use some coder convenience functions to create coder parameters and a dictionary of symbols to make available to the local dictionary

In [ ]:
params = coder.set_coder_params(param_strings,units)
symbols_dict = coder.get_symbol_dict_from_params(params)
locals().update(symbols_dict)

### Olivine melting reaction

Set up the forsterite melting reaction (Forsterite_berman -> Forsterite_xmelts). This requires a list of tuples for both the reactants and products. Each tuple consists of strings of 

(name,phase,endmember)

where 
    **name**:  is an arbitrary name describing the reactant endmenber,
    **phase**: is the phase containing the endmember
    **endmember**: the name of the endmember that is reacting in that phase
    
The phase and endmember names should be consistent with whatever they are called in the thermodynamic database.

In [ ]:
# reaction 0
j = 0

# Reactants
Fo_Ol = ('Fo_Ol','Olivine','Forsterite_berman')

# Products
Fo_Lq = ('Fo_Lq','Liquid','Forsterite_xmelts')

Now we write down the SymPy expression for the reaction. This, along with the list of reactants, products, parameters, units and variable symbols are passed into the 'add_reaction_to_model' function. This function also requires a 'name' field, which should be consistent with the variable assigned to the SymPy expression; for example, here our expression is Fo_melting = ..., so our name is 'Fo_melting'.

In [ ]:
# SymPy expression for reaction rate
# Rate constant
rp = r0_melt*sym.exp(-T0/T)
rm = r0_xtal*sym.exp(-T0/T)

# Availability
Sp = Stot*Phi[iOl]/(Phi[iOl] + eps)
Sm = Stot*Phi[iLq]/(Phi[iLq] + eps)

# Affinity function
fA = (M0/M_fo)*A[j]/R/T

Ol_melting = sym.Piecewise((rp*Sp*fA,A[0]>=0),(rm*Sm*fA,A[0]<0),(0,True))

In [ ]:
reactants = [ Fo_Ol ]
products = [ Fo_Lq ]
rxn.add_reaction_to_model('Ol_melting', reactants, products, Ol_melting, params)

### Enstatite melting reaction

In [ ]:
# Reaction 1
j = 1

# Reactants
En_Opx = ('En_Opx','Orthopyroxene','Orthoenstatite_berman')

# Products
#Fo_Lq = ('Fo_Lq','Liquid','Forsterite_xmelts')
Si_Lq = ('Si_lq', 'Liquid', 'Quartz4_liquid')

# Rate constant
rp = r0_melt*sym.exp(-T0/T)
rm = r0_xtal*sym.exp(-T0/T)

# Availability
Sp = Stot*Phi[iOpx]/(Phi[iOpx] + eps)
Sm = Stot*Phi[iLq]/(Phi[iLq] + eps)

# Affinity function
fA = (M0/M_en)*A[j]/R/T

Opx_melting = sym.Piecewise((rp*Sp*fA,A[j]>=0),(rm*Sm*fA,A[j]<0),(0,True))
Opx_melting

In [ ]:
reactants = [ En_Opx ] 
products = [ Fo_Lq, Si_Lq ]
rxn.add_reaction_to_model('Opx_melting', reactants, products, Opx_melting, params)

### Si melting reaction

In [ ]:
# Reaction 2
j = 2

# Reactants
Si_Si = ('Si_Si','Silica_polymorph','Silica_polymorph_berman')

# Products
#Si_Lq = ('Si_lq', 'Liquid', 'Quartz4_liquid')

# Rate constant
rp = r0_melt*sym.exp(-T0/T)
rm = r0_xtal*sym.exp(-T0/T)

# Availability
Sp = Stot*Phi[iSi]/(Phi[iSi] + eps)
Sm = Stot*Phi[iLq]/(Phi[iLq] + eps)

# Affinity function
fA = (M0/M_qz)*A[j]/R/T

Si_melting = sym.Piecewise((rp*Sp*fA,A[j]>=0),(rm*Sm*fA,A[j]<0),(0,True))
Si_melting

In [ ]:
reactants = [ Si_Si ] 
products = [ Si_Lq ]
rxn.add_reaction_to_model('Si_melting', reactants, products, Si_melting, params)

### The model_dict has now been updated to contain all of the information for these reactions

In [ ]:
rxn.model_dict

Return a dictionary of settable name value pairs

In [ ]:
values_dict = rxn.get_values()
values_dict

Update some of these values...

Here we will set the forward and backward reaction rates 

$$
    r_j^+=r_j^- =1
$$

and the total available reactive surface area $S_0=1$.  But these will be scaled using pre-factors and dimensionless Dahmkohler numbers in the geodynamics codes 


In [ ]:
values_dict.update(dict(name='fo_sio2_poly_linear_rxns',
                        reference=reference,
                        T0 = 1750.,
                        R = 8.314,
                        r0_xtal = 1.,
                        r0_melt = 1.,
                        Stot = 1.,
                        eps = 0.1,
                        M0 = 100.,
                        M_fo = 140.691,
                        M_en = 100.387,
                        M_qz = 60.083                  
                       ))
values_dict

... and update them in the model_dict using the 'set_values' function

In [ ]:
rxn.set_values(values_dict)
rxn.model_dict

### Generate Spud XML files

In [ ]:
rxn.to_xml(path=SPUD_DIR)